<a href="https://colab.research.google.com/github/kittimaxz/Project_BoneAge/blob/main/model_pytorchlightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 30.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from PIL import Image
from pytorch_lightning import seed_everything, LightningModule, Trainer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import scipy.stats as stats
import pytorch_lightning as pl
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from torch import nn, optim, rand, sum as tsum, reshape, save

In [3]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import datetime,os # datetime ใช้ในการจัด format ของวัน-เวลาให้อยู่ในรูปแบบที่ต้องการ 
import math # เรียกใช้ module เพื่อคำนวณค่าต่างๆ 
import torch.utils.checkpoint as cp
from collections import OrderedDict
import torch.utils.model_zoo as model_zoo
from torch.utils.model_zoo import load_url as load_state_dict_from_url

In [4]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [6]:
path = '/content/drive/My Drive/Project_Boneage'  # ชี้ว่า data เราอยู่โฟล์เดอร์ไหน

In [7]:
train_img = '/content/drive/My Drive/Project_Boneage/boneage_training_dataset/'
val_img = '/content/drive/My Drive/Project_Boneage/boneage_validation_dataset/'
test_img = '/content/drive/My Drive/Project_Boneage/boneage-test-dataset/'

In [8]:
train = pd.read_csv("/content/drive/MyDrive/Project_Boneage/train.csv")
validation = pd.read_csv("/content/drive/MyDrive/Project_Boneage/validation.csv")
test = pd.read_csv("/content/drive/MyDrive/Project_Boneage/boneage-test-dataset.csv")

In [9]:
train

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False
...,...,...,...
10084,13259,94,False
10085,13261,94,False
10086,13274,94,False
10087,13286,94,False


In [10]:
train['id'] = train['id'].apply(lambda x: str(x)+'.png')
train['male'] = train['male'].apply(lambda x: 0 if x else 1)

In [11]:
train['boneage zscore'] = stats.zscore(train['boneage'])

In [12]:
data_train = pd.DataFrame(train)
data_train.rename(columns={'male':'gender'},inplace=True)
data_train

,id,boneage,gender,boneage zscore
0,1377.png,180,1,1.278370
1,1378.png,12,1,-2.796477
2,1379.png,94,1,-0.807564
3,1380.png,120,0,-0.176933
4,1381.png,82,1,-1.098624
...,...,...,...,...
10084,13259.png,94,1,-0.807564
10085,13261.png,94,1,-0.807564
10086,13274.png,94,1,-0.807564
10087,13286.png,94,1,-0.807564


In [13]:
validation['id'] = validation['id'].apply(lambda x: str(x)+'.png')
validation['male'] = validation['male'].apply(lambda x: 0 if x else 1)

In [14]:
validation['boneage zscore'] = stats.zscore(validation['boneage'])

In [15]:
data_validation = pd.DataFrame(validation)
data_validation.rename(columns={'male':'gender'},inplace=True)
data_validation

,id,boneage,gender,boneage zscore
0,10186.png,10,0,-2.864934
1,10758.png,15,1,-2.742944
2,10860.png,180,1,1.282722
3,10877.png,180,1,1.282722
4,11008.png,180,1,1.282722
...,...,...,...,...
2517,15605.png,50,1,-1.889015
2518,15606.png,113,1,-0.351942
2519,15608.png,55,1,-1.767025
2520,15609.png,150,0,0.550783


In [16]:
test['Case ID'] = test['Case ID'].apply(lambda x: str(x)+'.png')
#test['boneage'] = test['boneage'].apply(lambda x: str(x))
#test['Sex'] = test['Sex'].apply(lambda x: 0 if x else 1)

data_test = pd.DataFrame(test)
data_test.rename(columns={'Sex':'gender'},inplace=True)
data_test['gender'].replace({ 'M':0,'F':1}, inplace=True)
data_test

,Case ID,gender,boneage
0,4360.png,0,169
1,4361.png,0,170
2,4362.png,0,73
3,4363.png,0,153
4,4364.png,0,135
...,...,...,...
195,4555.png,1,133
196,4556.png,1,129
197,4557.png,1,168
198,4558.png,1,136


In [17]:
class ImageDataset(Dataset):
    """Tabular and Image dataset."""

    def __init__(self, pickle_file, image_dir):
        self.image_dir = image_dir
        self.pickle_file = pickle_file
        self.tabular = pd.read_csv(pickle_file)

    def __len__(self):
        return len(self.tabular)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        tabular = self.tabular.iloc[idx, 0:]

        y = tabular["boneage"]

        image = Image.open(f"{self.image_dir}{tabular['id']}.png")
        image = np.array(image)
        #image = image[..., :3]

        image = transforms.functional.to_tensor(image)
        image = transforms.functional.resize(image, size=[500,500])
        # image = transforms.Compose([transforms.Resize((500, 500)),
        #                        transforms.RandomAffine(
        #                            20, translate=(0.2, 0.2),
        #                            scale=(1, 1.2)),
        #                        transforms.RandomHorizontalFlip(),
        #                        transforms.ToTensor()])

        tabular = tabular[["male"]]
        tabular = tabular.tolist()
        tabular = torch.FloatTensor(tabular)

        return image, tabular, y

In [18]:
image_datatrain = ImageDataset(pickle_file=path+"/train.csv", image_dir=f"{train_img}")
image_dataval = ImageDataset(pickle_file=path+"/validation.csv", image_dir=f"{val_img}")
image_datatest = ImageDataset(pickle_file=path+"/boneage-test-dataset.csv", image_dir=f"{test_img}")

In [19]:
strat_epoch =0 
num_epochs=60
lr=0.0001
patience=2 
batch_size=20
num_workers=0
num_gpus=1
gpus=0

In [20]:
train_dataloader = DataLoader(image_datatrain, batch_size,num_workers)
train_dataloader

In [21]:
val_dataloader = DataLoader(image_dataval, batch_size,num_workers)
val_dataloader

In [22]:
test_dataloader = DataLoader(image_datatest, batch_size,num_workers)
test_dataloader

In [23]:
! git clone https://github.com/BCV-Uniandes/Bonet.git
!cd Bonet

Cloning into 'Bonet'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (52/52), 136.62 KiB | 3.18 MiB/s, done.


In [24]:
# Local imports
from Bonet.models.backbone.inception_v3 import InceptionV3

In [25]:
class BoNetModule(pl.LightningModule):

    def __init__(
        self, transform_input = False, lr = 1e-3, num_workers = 4, batch_size = 32
    ):

        super().__init__()
        self.lr = lr
        self.num_workers = num_workers
        self.batch_size = batch_size
 
        # Backbone
        self.inception_v3 = InceptionV3()

        # Gender
        self.gender = DenseLayer(1, 32)
        self.fc_1 = DenseLayer(100384, 1000)
        self.fc_2 = DenseLayer(1000, 1000)
        self.fc_3 = nn.Linear(1000, 1)

    def forward(self, x, y):
        x = self.inception_v3(x)
        y = self.gender(y)
        x = self.fc_1(torch.cat([x, y], 1))
        x = self.fc_2(x)
        x = self.fc_3(x)
        return x

    def training_step(self, batch, batch_idx):
      image, tabular, y = batch

      criterion = torch.nn.L1Loss()
      y_pred = torch.flatten(self(image, tabular))
      y_pred = y_pred.double()

      loss = criterion(y_pred, y)
      return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        image, tabular, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(image, tabular))
        y_pred = y_pred.double()

        val_loss = criterion(y_pred, y)

        return {"val_loss": val_loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        return {"val_loss": avg_loss}

    def test_step(self, batch, batch_idx):
        image, tabular, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(image, tabular))
        y_pred = y_pred.double()

        test_loss = criterion(y_pred, y)

        return {"test_loss": test_loss}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        logs = {"test_loss": avg_loss}
        return {"test_loss": avg_loss, "log": logs, "progress_bar": logs}

    def setup(self, stage):

        image_datatrain = ImageDataset(pickle_file=path+"/train.csv", image_dir=f"{train_img}")
        image_dataval = ImageDataset(pickle_file=path+"/validation.csv", image_dir=f"{val_img}")
        image_datatest = ImageDataset(pickle_file=path+"/boneage-test-dataset.csv", image_dir=f"{test_img}")

        #train_size = int(0.80 * len(image_data))
        #val_size = int((len(image_data) - train_size) / 2)
        #test_size = int((len(image_data) - train_size) / 2)

        #self.train_set, self.val_set, self.test_set = random_split(image_data, (train_size, val_size, test_size))
        self.train_set = image_datatrain
        self.val_set = image_dataval
        self.test_set = image_datatest
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=(self.lr))

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)
    

class DenseLayer(pl.LightningModule):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.linear = nn.Linear(in_channels, out_channels)

    def forward(self, x):
        x = self.linear(x)
        return F.relu(x, inplace=True)

In [26]:
#%reload_ext tensorboard
#logs_dir = '.\logs'
#%tensorboard --logdir {logs_dir}

In [27]:
#logdir = os.path.join(logs_dir,datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))

In [28]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2 )

In [29]:
callbacks  = [ModelCheckpoint(
    monitor="valid_loss",
    dirpath="./pl-checkpoints",
    filename="sample-{epoch:02d}-{valid_loss:.03f}-{valid_accuracy:.03f}",
    save_top_k=1,
    mode="min",
)]

In [30]:
logger = CSVLogger(save_dir="logs/",version = 'version_0', name="my-model", flush_logs_every_n_steps=1)

In [31]:
#logger = pl.loggers.TensorBoardLogger(save_dir = "logdir",version = 'version_0', name="my_model")

In [32]:
model = BoNetModule()

trainer = pl.Trainer(logger = logger, accelerator='gpu', devices=1,max_epochs=4)
trainer.fit(model=model,train_dataloaders=train_dataloader,val_dataloaders=val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type        | Params
---------------------------------------------
0 | inception_v3 | InceptionV3 | 21.8 M
1 | gender       | DenseLayer  | 64    
2 | fc_1         | DenseLayer  | 100 M 
3 | fc_2         | DenseLayer  | 1.0 M 
4 | fc_3         | Linear      | 1.0 K 
---------------------------------------------
123 M     Trainable params
0         Non-trainable params
123 M     Total params
492.688   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
<ipython-input-17-4a3b2084aa90>:35: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  tabular = torch.FloatTensor(tabular)
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=4` reached.


In [33]:
metrics = pd.read_csv(f"{trainer.logger.log_dir}/metrics.csv")

aggreg_metrics = []
agg_col = "epoch"
for i, dfg in metrics.groupby(agg_col):
    agg = dict(dfg.mean())
    agg[agg_col] = i
    aggreg_metrics.append(agg)


FileNotFoundError: ignored

In [ ]:
df_metrics = pd.DataFrame(aggreg_metrics)

In [ ]:
df_metrics[["train_loss", "valid_loss"]].plot(
    grid=True, legend=True, xlabel='Epoch', ylabel='Loss')

In [ ]:
df_metrics[["train_acc", "valid_acc"]].plot(
    grid=True, legend=True, xlabel='Epoch', ylabel='ACC')

In [ ]:
#trainer.test()